In [24]:
# Internal
from pathlib import Path

# External
import numpy as np
import torch
import torch.nn as nn
import httpx

# project Configs
DATASET_PATH = Path("./data")
CHECKPOINT_PATH = Path("./models")
torch.backends.cudnn.benchmark=False
torch.backends.cudnn.deterministic=True
DEVICE = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

In [25]:
def set_seed(seed=42):
  np.random.seed(seed)
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
set_seed(42)

In [26]:
base_url = "https://raw.githubusercontent.com/phlippe/saved_models/main/tutorial3/"
# Files to download
pretrained_files = ["FashionMNIST_elu.config", "FashionMNIST_elu.tar",
                    "FashionMNIST_leakyrelu.config", "FashionMNIST_leakyrelu.tar",
                    "FashionMNIST_relu.config", "FashionMNIST_relu.tar",
                    "FashionMNIST_sigmoid.config", "FashionMNIST_sigmoid.tar",
                    "FashionMNIST_swish.config", "FashionMNIST_swish.tar",
                    "FashionMNIST_tanh.config", "FashionMNIST_tanh.tar"]

In [27]:
if not DATASET_PATH.exists():
  DATASET_PATH.mkdir(parents=True, exist_ok=True)

for file_path in pretrained_files:
  file_url = base_url+file_path
  file_path = DATASET_PATH / file_path
  response = httpx.get(str(file_url))
  with open(file_path, "wb") as f:
    f.write(response.content)

In [28]:
class ActivationFunction(nn.Module):

  def __init__(self) -> None:
    super().__init__()
    self.name = self.__class__.__name__
    self.config = {"name":self.name}

In [29]:
class Sigmoid(ActivationFunction):
  def forward(self,x):
    return 1/(1+torch.exp(-x))

class Tanh(ActivationFunction):
  def forward(self,x):
    x_exp,neg_x_exp = torch.exp(x), torch.exp(-x)
    return (x_exp - neg_x_exp)/(x_exp + neg_x_exp)